# Default Import

In [1]:
from ris import pysqldb
from IPython.display import Markdown, clear_output
from sqlalchemy import create_engine
import ris
import getpass
import datetime 
import pandas as pd
import numpy as np
import os
import requests

timestamp = datetime.datetime.now().strftime('%Y-%m-%d %H:%M')
ts = datetime.datetime.now().strftime('%Y-%m-%d')

Markdown('<strong>Notebook run on: {} | by {} | Using ris library version: {} | File Location: {}'.format(
    timestamp, getpass.getuser(), ris.__version__, os.getcwd()
))


<strong>Notebook run on: 2021-01-05 12:37 | by soge | Using ris library version: 2.0.1 | File Location: E:\RIS\Staff Folders\Samuel\Requests\SH\Crash_Requests\ped_veh_crashes

# Db connections

In [2]:
try: 
    db.params['user']
except:
    db = pysqldb.DbConnect(type='PG', server='dotdevrhpgsql01', database='ris')


Additional database connection details required:
User name (ris):soge
Password (ris)········


In [3]:
#sip = pysqldb.DbConnect(server='dotpgsql01', database='sip', type='PG', user = db.params['user'], password = db.params['password'])
#forms = pysqldb.DbConnect(server='dot55sql01', database='forms', type='MS', user = 'arcgis', password = 'arcgis')

In [4]:
data = db.dfquery("""

WITH data as (
SELECT distinct nys_a.crashid, nys_p.crashid ped, nys_v.crashid veh, nys_v.veh_cnt, nys_a.accd_type_int, case_yr
FROM public.nysdot_all nys_a
LEFT JOIN (SELECT distinct crashid
       FROM public.nysdot_vehicle
       WHERE case_yr> 2012
       AND veh_typ = '6'
       ) nys_p
     on nys_a.crashid = nys_p.crashid

LEFT JOIN (SELECT distinct crashid, count(veh_seq_num) veh_cnt
           FROM public.nysdot_vehicle
           WHERE case_yr>2012
           AND veh_typ not in ('5','6')
           GROUP BY crashid
           HAVING count(veh_seq_num)>1
           ORDER BY crashid
       ) nys_v
     on nys_a.crashid = nys_v.crashid
WHERE nys_a.case_yr>=2012
and nys_p.crashid is not null
and nys_v.crashid is not null
--and exclude = 0
)

SELECT case_yr
      ,sum(CASE WHEN accd_type_int = 1 THEN 1 END)::decimal/count(DISTINCT crashid)*100 "accd_type_int=1"
      ,sum(CASE WHEN accd_type_int = 2 THEN 1 END)::decimal/count(DISTINCT crashid)*100 "accd_type_int=2"
      ,sum(CASE WHEN accd_type_int = 3 THEN 1 END)::decimal/count(DISTINCT crashid)*100 "accd_type_int=3"
FROM data
GROUP BY case_yr

""")

In [5]:
data

,case_yr,accd_type_int=1,accd_type_int=2,accd_type_int=3
0,2013,40.05102040816326530600,0.25510204081632653100,59.69387755102040816300
1,2014,38.29787234042553191500,0.53191489361702127700,61.17021276595744680900
2,2015,37.37373737373737373700,None,62.62626262626262626300
3,2016,40.28985507246376811600,0.28985507246376811600,59.42028985507246376800
4,2017,38.37837837837837837800,None,61.62162162162162162200
5,2018,38.43750000000000000000,None,61.56250000000000000000


In [6]:
data.to_excel('ped_veh_crashes_2013_now.xlsx')